TODO: look at pg 14 of: https://arxiv.org/pdf/quant-ph/0406176v5.pdf

This is what Qiskit uses

https://quantumcomputing.stackexchange.com/questions/6755/controlled-initialize-instruction

Code here implements the algorithm in: https://arxiv.org/pdf/quant-ph/0104030.pdf

In [1]:
from quchem.Qcircuit.arb_state_prep_circuit_functions import My_U_Gate

import numpy as np
import cirq

In [2]:
theta = np.pi

U_single_qubit = My_U_Gate(theta)
op = U_single_qubit.on(cirq.LineQubit(1))
print(cirq.Circuit(op))

print('####')

op = U_single_qubit.controlled(num_controls=3, control_values=[0, 0, 1]).on(*[cirq.LineQubit(1), cirq.LineQubit(2), cirq.LineQubit(3)], cirq.LineQubit(4))
print(cirq.Circuit(op))

1: ─── U = 3.1416 rad ───
####
1: ───(0)────────────────
      │
2: ───(0)────────────────
      │
3: ───@──────────────────
      │
4: ─── U = 3.1416 rad ───


In [3]:
from quchem.Qcircuit.arb_state_prep_circuit_functions import Get_arb_state_prep_circuit_params

N_qubits=3
state_vector = [1/np.sqrt(2**N_qubits) for _ in range(2**N_qubits)]

alpha_j_dict = Get_arb_state_prep_circuit_params(N_qubits, state_vector)
alpha_j_dict

{0: [{'control_state': '', 'angle': 0.7853981633974483}],
 2: [{'control_state': '00', 'angle': 0.7853981633974483},
  {'control_state': '01', 'angle': 0.7853981633974483},
  {'control_state': '10', 'angle': 0.7853981633974483},
  {'control_state': '11', 'angle': 0.7853981633974483}],
 1: [{'control_state': '0', 'angle': 0.7853981633974483},
  {'control_state': '1', 'angle': 0.7853981633974483}]}

In [4]:
from quchem.Qcircuit.arb_state_prep_circuit_functions import State_Prep_Circuit

N_system_qubits=0
circ_obj = State_Prep_Circuit(alpha_j_dict, N_system_qubits=N_system_qubits)
circuit = (
    cirq.Circuit(cirq.decompose_once((circ_obj(*cirq.LineQubit.range(N_system_qubits, N_system_qubits+circ_obj.num_qubits()))))))
circuit

0: ───H───(0)───@───(0)───(0)───@─────@───
          │     │   │     │     │     │
1: ───────H─────H───(0)───@─────(0)───@───
                    │     │     │     │
2: ─────────────────H─────H─────H─────H───

In [5]:
print(np.allclose(np.array(circuit.unitary()[:,0]),
   Get_arb_state_prep_circuit_params         np.array(state_vector)))

np.array(circuit.unitary())[:,0]

True


array([0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j,
       0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j])

In [6]:
from quchem.Qcircuit.arb_state_prep_circuit_functions import Get_G_and_Gdag_circuits
N_system_qubits=2

G_circuits = Get_G_and_Gdag_circuits(state_vector, N_system_qubits, check_state_prep_circuit=True)

G_circuits.G_circuit

2: ───H───(0)───@───(0)───(0)───@─────@───
          │     │   │     │     │     │
3: ───────H─────H───(0)───@─────(0)───@───
                    │     │     │     │
4: ─────────────────H─────H─────H─────H───

In [8]:
G_circuits.G_DAGGER_circuit

2: ───@───@─────(0)───(0)───@───(0)───H───
      │   │     │     │     │   │
3: ───@───(0)───@─────(0)───H───H─────────
      │   │     │     │
4: ───H───H─────H─────H───────────────────

In [32]:
from quchem.Qcircuit.arb_state_prep_circuit_functions import Get_G_and_Gdag_circuits
N_system_qubits=2

state_vector = [np.sqrt(0.4), np.sqrt(0.3), np.sqrt(0.2), np.sqrt(0.1)]

G_circuits = Get_G_and_Gdag_circuits(state_vector, N_system_qubits, check_state_prep_circuit=False)

G_circuits.G_circuit

2: ─── U = 0.5796 rad ───(0)────────────────@──────────────────
                         │                  │
3: ────────────────────── U = 0.7495 rad ─── U = 0.6992 rad ───

In [33]:
G_circuits.G_circuit.unitary()[0,:]

array([0.61247765+0.j, 0.56997468+0.j, 0.40096074+0.j, 0.37313601+0.j])

In [34]:
state_vector = [np.sqrt(0.4), np.sqrt(0.3), np.sqrt(0.2), np.sqrt(0.1)]
alpha_j_dict = Get_arb_state_prep_circuit_params(2, state_vector)
alpha_j_dict

{0: [{'control_state': '', 'angle': 0.5796397403637044}],
 1: [{'control_state': '0', 'angle': 0.7494688654174801},
  {'control_state': '1', 'angle': 0.6991851645410239}]}

In [36]:
state_vector

[0.6324555320336759,
 0.5477225575051661,
 0.4472135954999579,
 0.31622776601683794]